# 챗봇 만들기

### https://jfun.tistory.com/199?category=851249
### https://www.slideshare.net/KimSungdong1/20170227-72644192

In [1]:
input_data = '판교에 지금 주문해줘'
output_data = ''

request = {
    "intent_id":"",
    "input_data":input_data,
    "request_type":"text",
    "story_slot_entity":{},
    "output_data":output_data
}

## 기본 데이터 셋(DB))

In [2]:
intent_list = {
    "주문":["주문", "배달"],
    "예약":["예약", "잡아줘"],
    "정보":["정보","알려"]
}

story_slot_entity = {"주문":{"메뉴":None, "장소":None, "날짜":None},
"예약":{"장소": None, "날짜":None},
"정보":{"대상":None}
}

## 형태소 분석

In [3]:
from eunjeon import Mecab
mecab = Mecab()
preprocessed = mecab.pos(request.get('input_data'))

print(preprocessed)

[('판교', 'NNG'), ('에', 'JKB'), ('지금', 'MAG'), ('주문', 'NNG'), ('해', 'XSV+EC'), ('줘', 'VX+EC')]


## Intent 도출(Ruled Based)

In [4]:
# Char CNN을 사용해서 연결하면 좀 더 쉽게 만들 수 있다
intent_id = "주문"
slot_value = story_slot_entity.get("주문")

## NER 도출(Ruled based)

In [5]:
# LSTM 기법을 사용해서 연결하면 좀 더 쉬울 수 있다
menu_list = ["피자", "햄버거", "치킨"]
loc_list = ["판교", "야탑", "서현"]
date_list = ["지금", '내일', '모레']

## Dictionary 기반 slot 구성

In [12]:
for pos_tag in preprocessed:
    if pos_tag[1] in ['NNG', 'NNP', 'SL', 'MAG']:
        if pos_tag[0] in menu_list:
            slot_value['메뉴'] = pos_tag[0]
        elif pos_tag[0] in loc_list:
            slot_value['장소'] = pos_tag[0]
        elif pos_tag[0] in date_list:
            slot_value['날짜'] = pos_tag[0]


print(story_slot_entity.get('주문'))

{'메뉴': None, '장소': '판교', '날짜': '지금'}


## 빈 slot 검색